In [ ]:
import pandas as pd
import numpy as np
import os

# Format: 20 input and 10 output with position, velocity, and acceleration
# taken at every 10th of a second interval, so there are over lapping datapoints, however we can change it to our liking

# right now all of the files are in the same directory as this file, so we can use the "."
# however if we are parsing through different directories we can just change it to be in the one that we want
path = '../data/random_trajectory_100ms/'

# files_content = []

master = pd.DataFrame(columns=['input', 'output'])
# for each txt file in the directory
for filename in filter(lambda p: p.endswith("txt"), os.listdir(path)):
    # read the filename
    df = pd.read_csv(filename)
    print(filename)
    
    # for now only use the position and timestamps for the velocity
    df = df[['timestamp', 'tx', 'ty', 'tz']]

    # get the difference in time stamps
    # it is actually the same in both unix time conversions and the synthetic data :D
    df['dt'] = df['timestamp'].diff()
    # make sure that ther are no division by zero for the velocity calculation
    df['dt'] = df['dt'].replace(0, np.nan)

    # calculte the velocity vectors for the three different directions
    df['vx'] = df['tx'].diff() / df['dt']
    df['vy'] = df['ty'].diff() / df['dt']
    df['vz'] = df['tz'].diff() / df['dt']

    # calculate the acceleration vectors for the three different directions
    df['ax'] = df['vx'].diff() / df['dt']
    df['ay'] = df['vy'].diff() / df['dt']
    df['az'] = df['vz'].diff() / df['dt']
    
    # we are creating a dataframe 
    for i in range(len(df) - 30):
        input = []
        output = []
        for j in range(20):
            temp = df.iloc[i+j]
            input.append([0.1*j, temp["tx"], temp["ty"], temp["tz"], temp["vx"], temp["vy"], temp["vz"], temp["ax"], temp["ay"], temp["az"]])
            if j < 10:
                temp = df.iloc[i+j+20]
                output.append([(0.1*j)+2, temp["tx"], temp["ty"], temp["tz"], temp["vx"], temp["vy"], temp["vz"], temp["ax"], temp["ay"], temp["az"]])
        master.loc[len(master)] = [input, output]

master.to_csv("combinedTrajectories.csv", index=False)